# Modul 12: Bilder

I denne modulen skal vi gjennomføre en øvelse som involverer bilder. I de ulike ingeniørfaglige områdene blir bilder brukt overalt. Bilder inkluderer fotografier, satellittbilder og flyfotografier.

Bilder er i hovedsak lag med rutenettdata med en intensitetsverdi ved hvert piksel. Lagene i bildet kalles kanaler (for eksempel røde, grønne og blå kanaler), og kombinasjonen av disse kanalene produserer en spesifikk farge ved hvert piksel.

Dette eksempelet illustrerer lesing, manipulering og lagring av bilder.

Men først må du installere noen ting...

## Hva du trenger å installere
I Python kan vi bruke pillow-biblioteket for å laste, vise og manipulere bilder. Vi tror pillow er installert i Anaconda-distribusjonen, men hvis ikke, må du installere det slik:

In [ ]:
# run this cell if pillow is not installed
import sys
!{sys.executable} -m pip install pillow

### Importer ditt eget bilde
Vi importerer vårt bilde: "SURF.JPG". OBS: Du må sørge for at bildet du velger å importere, er i samme mappe som notatboken din.

In [ ]:
# importer og print bildet

from PIL import Image # importerer bilde ved hjelp av PIL biblioteket
import os

im = Image.open(os.path.join("SURF.JPG")) # importer dit bilde istedenfor "SURF.JPG"

print(im.format, im.size, im.mode) # print ut informasjonen om bildet

im

Bildet er i JPEG-format, og det har 6000 x 3376 piksler, fordelt på tre kanaler: rød (R), grønn (G) og blå (B). Nå skal vi konvertere bildet til en matrise ved hjelp av `numpy.array`-metoden, og skrive ut formen, samt maksimale og minimale verdier i matrisen:

In [ ]:
import numpy as np

data = np.array(im) # konverter bildet til array

print(data.shape) # Skriv ut form

print("minimum value =", np.amin(data), "maximum value =", np.amax(data)) # Printer ut min og max array-verdier

Dette er en 3D-matrise, som representerer 3 (`data.shape[2]`) 2D-lag med rutenett (hver på størrelse 3376 x 6000 piksler), tilsvarende R, G og B-kanalene. Minimums- og maksimumsverdiene i rutenettet er 0 og 255. Disse verdiene kommer fra bildets format (JPG), som bruker et fast 8-bits fargekart med $2^8=256$ mulige kombinasjoner som representerer ulike intensiteter. Nå skal vi plotte bildet:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 6)) # create figure

ax.imshow(data); # plot image

La oss nå se på farge kanalene i et histogram!

In [ ]:
farger = ["red", "green", "blue"] # Liste over farger
kanal_id = [0, 1, 2] # Liste over kanal-ID-er

fig, ax = plt.subplots() # Lager figur

# plotter histogram
for kanal_id, c in zip(kanal_id, farger): # Går over hver kanal id og farge
    histogram, bin_edges = np.histogram(data[:, :, kanal_id], bins=256, range=(0, 256)) # lager histogrammet
    ax.plot(bin_edges[0:-1], histogram, color=c, label=c + " channel") # plotter histogrammet

# figure tittel, legende og akse-navn.
ax.set_title("Histogram Surf image")
ax.set_xlabel("verdier")
ax.set_ylabel("pixler")
ax.legend(loc = "upper left");

Her er histogrammene for de individuelle fargekanalene.

La oss nå gjøre noe mer interessant. La oss behandle bildet for å bedre fremheve hovedkomponentene. For å gjøre dette bruker vi [scikit-image](https://scikit-image.org)-biblioteket, som er en samling av algoritmer for bildebehandling. Vi tror at `scikit-image` er installert i Anaconda-distribusjonen, men hvis det ikke er det, må du installere det slik:

In [ ]:
# Kjør denne cellen hvis du ikke har scikit-image installert
import sys
!{sys.executable} -m pip install scikit-image

La oss konvertere bildet til gråtone ved å bruke funksjonen `scikit-image.color.rgb2gray` (mer om hvordan denne funksjonen fungerer [her](https://scikit-image.org/docs/dev/auto_examples/color_exposure/plot_rgb_to_gray.html)), og deretter gjøre bildet litt mykere ved hjelp av en gaussisk filter ([sckit-image.filters.gaussian](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian)). Vi skriver også ut størrelsen på det filtrerte bildet, minimums- og maksimumsverdiene, og plotter bildet:

In [ ]:
from skimage import color, filters

data_gray = color.rgb2gray(data) # konverterer bilde to en enkel gråskala kanal

data_grad = filters.gaussian(data_gray) # bruker Gaussian filter for gråskala bilde

# printer ut størrelsen på min og max verdi
print(data_grad.shape) 
print("minimum value = {:.3f}, maximum value = {:.3f}".format(np.amin(data_grad), np.amax(data_grad)) ) 

# vis gråskala bildet. Du kan også prøve 'viridis' på cmap
fig, ax = plt.subplots(figsize=(8, 6))
plt.imshow(data_grad, cmap="gray")# bruker 'gray' på cmap for gråskala
cbar= plt.colorbar();# grå skala bar
cbar.set_label('Intensity', rotation=270, labelpad=10)

Nå skal vi prøve å gjøre litt forskjellige ting med bildet:

In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
from IPython.display import display

def process_and_display(input_path):
    # Åpne bildet
    img = Image.open(input_path)

    # Vis det orginale bildet
    print("Originalbilde:")
    display(img)

    # Gjør bildet uskarpt
    img_blurred = img.filter(ImageFilter.GaussianBlur(radius=10))#adjust the radius for more blur
    print("Uskarpt bilde:")
    display(img_blurred)

    # Gjør bildet skarpt
    img_sharpened = img.filter(ImageFilter.UnsharpMask(radius=3, percent=300, threshold=3))
    print("Skarpere bilde:")
    display(img_sharpened)

    # Endrer lysstyrke
    enhancer_brightness = ImageEnhance.Brightness(img)
    img_brightness = enhancer_brightness.enhance(1.5)  # Adjust the factor as needed
    print("Lysere bilde:")
    display(img_brightness)

    # Endre kontrast nivå
    enhancer_contrast = ImageEnhance.Contrast(img)
    img_contrast = enhancer_contrast.enhance(1.5)  # Adjust the factor as needed
    print("Bilde med høy kontrast:")
    display(img_contrast)

# Erstatt denne veien med din vei
input_image_path = os.path.join("SURF.JPG")
    
# Bruk funksjonen
process_and_display(input_image_path)

Det var alt for denne gang. Dette er kun en kort introduksjon til hvordan du kan bruke bilder i python, det er mange flere muligheter også!